In [ ]:
import geopandas as gpd
import pandas as pd
import parquet

In [ ]:
wells = pd.read_csv("raw_data/cleaned_well_data.csv")

# wells = pd.read_csv("raw_data/Well_Registry.csv")
wells['date'] = pd.to_datetime(wells['date']) 
wells.info()

In [ ]:
wells = wells[wells["date"]>="1/1/2000"]
wells = wells[wells["date"]<="1/1/2020"]
wells["utc_seconds"] = pd.to_datetime(wells['date']).astype(int)/ 10**9

In [ ]:
wells = wells[wells["depth"].notnull()]
wells.info()

In [ ]:
well1 = wells[wells["wellid"]==343957112322001]
well1

In [ ]:
import numpy as np
well1
df_unix_sec = pd.to_datetime(well1['date']).astype(int)/ 10**9
f = np.polyfit(well1["utc_seconds"], well1['depth'], deg=1)

# Slope
f[0]*(60*60*24*365*20)

In [ ]:
def calculate_trend_slope(df):
    f = np.polyfit(df["utc_seconds"], df['depth'], deg=1)
    # Slope
    return f[0]*(60*60*24*365*20)


In [ ]:
wells = wells.groupby('wellid').agg({
    'SITE_LONGIT_DECIMAL':'first',
    'SITE_LATITUDE_DECIMAL':'first',
    'depth': lambda x: list(x),
    'utc_seconds': lambda x: list(x)
    }).reset_index()

In [ ]:
wells

In [ ]:
wells["depth_20_year_trend"] = wells.apply(calculate_trend_slope, axis=1)
wells

In [ ]:
wells[wells["wellid"]==343957112322001]

In [ ]:
wells.to_parquet("our_data/wells_with_depth_trend.parquet")